In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import matplotlib.pyplot as plt
import warnings
import model_report as mr
import pickle
warnings.filterwarnings('ignore')
%matplotlib inline


In [2]:
file_info = "C:\\Users\\blgai\\OneDrive\\Documents\\School\\SMU\\Courses\\Fall 2021\\Capstone A\Data\\cleaned_chunked_v2.csv"
df = pd.read_csv(file_info, sep='\t')

In [3]:
df.groupby(df.category).size()

category
bu    11731
dr     1595
ed     3894
mk     8775
os     6789
sp     2535
sx     6557
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
#get 10% holdout set
train_corpus, holdout_corpus, train_label, holdout_label = train_test_split(df['clean_comp_nouns'], 
                                                                           df['category'],
                                                                           test_size=.10,random_state=1234)

#get 80% train and 20% test sets
train_corpus, test_corpus, train_label, test_label = train_test_split(train_corpus,
                                                                      train_label,
                                                                      test_size=.2225,random_state=1234) #.9 * .2225 = .20

train_corpus.shape, test_corpus.shape, holdout_corpus.shape

((29302,), (8386,), (4188,))

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

#build BOW features on train corpus
tv = TfidfVectorizer(use_idf=True, min_df=25,max_df=.9, norm="l2",smooth_idf=True)
tv_train_features = tv.fit_transform(train_corpus)
#transform test corpus into features
tv_test_features = tv.transform(test_corpus)
#transform holdout corpus into features
tv_holdout_features = tv.transform(holdout_corpus)

print('TFIDF model:> Train features shape:',tv_train_features.shape,'Test features shape:',tv_test_features.shape,'Holdout features shape:',tv_holdout_features.shape)

TFIDF model:> Train features shape: (29302, 17587) Test features shape: (8386, 17587) Holdout features shape: (4188, 17587)


In [7]:
#Let's find the top important features and analyze them

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X_opt=SelectKBest(chi2, k=4100)
tv_train_features_trimmed = X_opt.fit_transform(tv_train_features, train_label)
tv_train_features_trimmed.shape

(29302, 4100)

In [8]:
#create a data frame that includes all features and whether or not they are deemed important by feature_selection
feat_imp = pd.DataFrame(list(zip(tv.get_feature_names(),X_opt.get_support().tolist())),columns = ['Features','Important'])
feat_imp[feat_imp.Important == True]

,Features,Important
2,aaliyah,True
11,abby,True
15,abella,True
17,abigail,True
27,abroad,True
...,...,...
17540,zamnesia,True
17542,zapp,True
17554,zen,True
17569,zoe,True


In [9]:
#Now we can grab the important features so that we can select only the important features from the sparse matrix created by TfidfVectorizer
imp_Feats_array = np.array(feat_imp[feat_imp.Important == True].index)

In [10]:
#Now we can pull only the important features out of the original train, test, and holdout matrices
tv_train_features_sub = tv_train_features[:,imp_Feats_array]
tv_test_features_sub = tv_test_features[:,imp_Feats_array]
tv_holdout_features_sub = tv_holdout_features[:,imp_Feats_array]

#Examine the new shapes of the sub-selected features
print('TFIDF model:> Train features shape:',tv_train_features_sub.shape,'Test features shape:',tv_test_features_sub.shape,'Holdout features shape:',tv_holdout_features_sub.shape)

TFIDF model:> Train features shape: (29302, 4100) Test features shape: (8386, 4100) Holdout features shape: (4188, 4100)


In [11]:
#setting up the classifiers
clf1 = LogisticRegression(penalty='l2', max_iter=100,C=1,random_state=1234)
clf2 = svm.SVC(kernel='linear', C=1.0, random_state=1234)
clf3 = SGDClassifier(penalty='l2', learning_rate='optimal', loss='hinge', n_jobs=1, early_stopping=True, n_iter_no_change=5, verbose=1,random_state=1234)
clf4 = svm.SVC(kernel='rbf', C=1.0, random_state=1234)

#setting up the voting classifier
ensemble_clf = VotingClassifier(estimators=[('lr',clf1),('svm',clf2),('sgd',clf3),('svm_r',clf4)],voting='hard')

In [12]:
#params to use in grid search
params = {'weights':[(1,1,1,1),(2,1,1,1),(1,2,1,1),(1,1,2,1),(1,1,1,2),(1,2,1,2),(1,1,2,2)]}

#find the optimal weights
grid = GridSearchCV(ensemble_clf,params)
grid.fit(tv_train_features_sub,train_label)
grid.best_params_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-- Epoch 1
Norm: 32.15, NNZs: 3734, Bias: -0.773412, T: 21096, Avg. loss: 0.214799
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 30.45, NNZs: 3806, Bias: -0.783441, T: 42192, Avg. loss: 0.186152
Total training time: 0.02 seconds.
-- Epoch 3
Norm: 30.17, NNZs: 3851, Bias: -0.751492, T: 63288, Avg. loss: 0.181542
Total training time: 0.03 seconds.
-- Epoch 4
Norm: 29.93, NNZs: 3863, Bias: -0.765386, T: 84384, Avg. loss: 0.178710
Total training time: 0.04 seconds.
-- Epoch 5
Norm: 29.90, NNZs: 3868, Bias: -0.728547, T: 105480, Avg. loss: 0.177933
Total training time: 0.05 seconds.
-- Epoch 6
Norm: 29.89, NNZs: 3874, Bias: -0.751618, T: 126576, Avg. loss: 0.177030
Total training time: 0.07 seconds.
Convergence after 6 epochs took 0.07 seconds
-- Epoch 1
Norm: 12.50, NNZs: 3004, Bias: -1.158023, T: 21096, Avg. loss: 0.012210
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 11.85, NNZs: 3243, Bias: -1.120968, T: 42192, Avg. loss: 0.009368
Total training time: 0.02 seconds.
-- Epoch 

[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-- Epoch 1
Norm: 31.84, NNZs: 3784, Bias: -0.792496, T: 21096, Avg. loss: 0.214916
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 30.48, NNZs: 3842, Bias: -0.770830, T: 42192, Avg. loss: 0.187718
Total training time: 0.02 seconds.
-- Epoch 3
Norm: 30.09, NNZs: 3893, Bias: -0.737654, T: 63288, Avg. loss: 0.182849
Total training time: 0.03 seconds.
-- Epoch 4
Norm: 30.03, NNZs: 3910, Bias: -0.757079, T: 84384, Avg. loss: 0.180239
Total training time: 0.04 seconds.
-- Epoch 5
Norm: 30.02, NNZs: 3914, Bias: -0.714043, T: 105480, Avg. loss: 0.178999
Total training time: 0.05 seconds.
-- Epoch 6
Norm: 29.90, NNZs: 3916, Bias: -0.742994, T: 126576, Avg. loss: 0.177996
Total training time: 0.07 seconds.
Convergence after 6 epochs took 0.07 seconds
-- Epoch 1
Norm: 12.34, NNZs: 2988, Bias: -1.146835, T: 21096, Avg. loss: 0.011813
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 11.71, NNZs: 3235, Bias: -1.114599, T: 42192, Avg. loss: 0.009445
Total training time: 0.02 seconds.
-- Epoch 

[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

-- Epoch 1
Norm: 32.51, NNZs: 3881, Bias: -0.767368, T: 21097, Avg. loss: 0.218090
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 30.69, NNZs: 3939, Bias: -0.793181, T: 42194, Avg. loss: 0.187108
Total training time: 0.02 seconds.
-- Epoch 3
Norm: 30.18, NNZs: 3947, Bias: -0.762368, T: 63291, Avg. loss: 0.182299
Total training time: 0.03 seconds.
-- Epoch 4
Norm: 30.08, NNZs: 3974, Bias: -0.777547, T: 84388, Avg. loss: 0.180250
Total training time: 0.04 seconds.
-- Epoch 5
Norm: 29.99, NNZs: 3977, Bias: -0.761068, T: 105485, Avg. loss: 0.178800
Total training time: 0.05 seconds.
-- Epoch 6
Norm: 29.91, NNZs: 3979, Bias: -0.777566, T: 126582, Avg. loss: 0.177821
Total training time: 0.06 seconds.
Convergence after 6 epochs took 0.07 seconds
-- Epoch 1
Norm: 12.86, NNZs: 2991, Bias: -1.155851, T: 21097, Avg. loss: 0.011979
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 11.93, NNZs: 3295, Bias: -1.150837, T: 42194, Avg. loss: 0.009143
Total training time: 0.02 seconds.
-- Epoch 

[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-- Epoch 1
Norm: 30.71, NNZs: 3795, Bias: -0.756367, T: 26371, Avg. loss: 0.212422
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 29.80, NNZs: 3894, Bias: -0.738174, T: 52742, Avg. loss: 0.188178
Total training time: 0.02 seconds.
-- Epoch 3
Norm: 29.50, NNZs: 3909, Bias: -0.770154, T: 79113, Avg. loss: 0.183494
Total training time: 0.04 seconds.
-- Epoch 4
Norm: 29.36, NNZs: 3936, Bias: -0.742107, T: 105484, Avg. loss: 0.181987
Total training time: 0.05 seconds.
-- Epoch 5
Norm: 29.30, NNZs: 3959, Bias: -0.740875, T: 131855, Avg. loss: 0.181120
Total training time: 0.07 seconds.
-- Epoch 6
Norm: 29.30, NNZs: 3962, Bias: -0.741231, T: 158226, Avg. loss: 0.180343
Total training time: 0.08 seconds.
Convergence after 6 epochs took 0.09 seconds
-- Epoch 1
Norm: 12.01, NNZs: 3084, Bias: -1.148003, T: 26371, Avg. loss: 0.011891
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 11.61, NNZs: 3284, Bias: -1.128004, T: 52742, Avg. loss: 0.009449
Total training time: 0.02 seconds.
-- Epoch

[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.6s finished


{'weights': (1, 1, 2, 1)}

In [13]:
grid.best_score_

0.8581667562466772